# EID Report

#### Please, select the period for which you want to perform the report:

In [1]:
period_1 = 'Q1'
year = 2023
start_date = '1er Octobre'

### Importing the required packages

In [2]:
#Importing packages

import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import pymysql
from sqlalchemy import create_engine
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_COLOR_INDEX
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR')

'fr_FR'

### Connecting to HIV Haiti and running the EID script

In [3]:
#connecting to the MySQL database

# ici, j'utilise les accréditations de la base de données HIV Haiti stockées dans un fichier environnemental 'dot.env'
load_dotenv('dot.env')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
db = os.getenv('MYSQL_DB')

conn_text = f'mysql+pymysql://{user}:{password}@{host}/{db}'
engine = create_engine(conn_text)

sql = '''
select Patient_code,Code_mere,if(Code_mere in (select patient_code from patient),'yes','no') as Liaison_mere,left(patient_code,8) as Site_or_lab_code, hospital,Office, if(Network is not null,Network,'Autres') as Network, date_of_birth, gender, start_date as arv_start, if(Office = 'CAP', 'HUJ', lab) as lab, date_blood_taken, date_caris_received_sample, date_lab_received_sample, Result,
if(timestampdiff(day,date_of_birth,date_blood_taken)<=61,if(timestampdiff(day,date_of_birth,date_blood_taken)>=0,'0_2',''),if(timestampdiff(day,date_of_birth,date_blood_taken)<=365,if(timestampdiff(day,date_of_birth,date_blood_taken)>61,'2_12',''),'')) as tranche_age
,if(arv is not null,'yes','') as on_arv, timestampdiff(day,date_lab_received_sample,now()) as total_jours_Labo, timestampdiff(day,date_blood_taken,pcr_result_date) as from_specimen_to_result, timestampdiff(day,date_blood_taken,date_caris_received_sample) as from_specimen_to_caris,
timestampdiff(day,date_caris_received_sample,date_lab_received_sample) as from_caris_to_lab, timestampdiff(day,date_lab_received_sample,pcr_result_date) as from_lab_to_result,

if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab")) as days_in_lab,
case 
	when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))>15 then 'greater than 15'
    when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))<=15 then 'less or equal to 15'
    else'not in lab' end as Nbres_jours_Labo
   , 
   #timestampdiff(day,date_caris_received_sample,now()) as Total_jours_Caris,
   
if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) as days_between_caris_lab,
case 
	when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00') ,timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) >5 then 'greater than 5'
    when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) <=5 then 'less or equal to 5'
 end as Nbres_jours_Caris,

if(pcr_result=1,if(patient_info is not null,if(is_abandoned=0 and is_dead=0,'no','yes'),''),'') as pos_dead_abandon
,death_date
,abandoned_date
,pcr_result_date,
case 
    when month(date_blood_taken) between 10 and 12 then 'Q1'
    when month(date_blood_taken) between 1 and 3 then 'Q2'
    when month(date_blood_taken) between 4 and 6 then 'Q3'
    else 'Q4'
 End as Quarter,  -- month(date_blood_taken)  as month_blood_taken,
 #year(date_blood_taken) as year_test,
if( month(date_blood_taken) between 10 and 12 ,year(date_blood_taken)+1,year(date_blood_taken) ) as Year
from(
select patient_code,tme.id_patient as id_in_mere ,concat(tme.mother_city_code,'/',tme.mother_hospital_code,'/',tme.mother_code) as Code_mere, date_of_birth, tracking_infant.id_patient as patient_info, date_blood_taken, pcr_result,arv.id_patient as arv, is_dead, is_abandoned, death_date, abandoned_date, pcr_result_date, date_caris_received_sample, date_lab_received_sample,
lookup_testing_specimen_result.name as _result, if(lookup_testing_specimen_result.name="", 'En cours',lookup_testing_specimen_result.name) as Result,lookup_hospital.name as hospital,lookup_network.name as Network,lookup_hospital.office as Office, lookup_testing_gender.name as gender, tracking_regime.start_date, lookup_lab.name as lab from(
    select * from testing_specimen t1 where timestampdiff(day,date_of_birth,date_blood_taken)<=365 and date_blood_taken between '2014-10-01' and '2030-12-31'
    and date_blood_taken=(select min(date_blood_taken) from testing_specimen t2 where t1.id_patient=t2.id_patient)
)test_1
left join patient pa on pa.id=test_1.id_patient
left join testing_mereenfant tme on pa.id= tme.id_patient
left join
(select * from(
    select id_patient from tracking_followup where on_arv=1
    union
    select id_patient from tracking_regime where category='regime_infant_treatment'
    )o
)arv on arv.id_patient=test_1.id_patient
left join tracking_infant on tracking_infant.id_patient=test_1.id_patient
left join lookup_testing_specimen_result on pcr_result=lookup_testing_specimen_result.id
left join lookup_hospital on concat(lookup_hospital.city_code,'/',lookup_hospital.hospital_code)=left(patient_code,8)
left join tracking_regime on tracking_regime.id_patient=test_1.id_patient
left join lookup_testing_gender on lookup_testing_gender.id=gender
left join lookup_lab on lookup_hospital.id_lab=lookup_lab.id
left join lookup_network on lookup_hospital.network=lookup_network.id
where timestampdiff(day,test_1.date_of_birth,test_1.date_blood_taken)<=365 and test_1.date_blood_taken between '2014-10-01' and '2030-12-31' and linked_to_id_patient=0
group by test_1.id_patient) final
'''

eid = pd.read_sql_query(sql, engine, parse_dates = True)

#fermeture de la connexion
engine.dispose()

#-------------------------------------------------------------------------------------------------------------------

#Renommer la colonne 'Site_or_lab_code'
eid.rename(columns = {'Site_or_lab_code' : 'CODE LABO'}, inplace = True)

eid.head(2)

,Patient_code,Code_mere,Liaison_mere,CODE LABO,hospital,Office,Network,date_of_birth,gender,arv_start,...,days_in_lab,Nbres_jours_Labo,days_between_caris_lab,Nbres_jours_Caris,pos_dead_abandon,death_date,abandoned_date,pcr_result_date,Quarter,Year
0,CAP/CDSF/DJ0815D,CAP/CDSF/01674,no,CAP/CDSF,CDS La Fossette,CAP,Santé,2015-08-31,None,None,...,23,greater than 15,22,greater than 5,,None,None,2015-12-04,Q1,2016
1,PAP/FOND/00000093,None,no,PAP/FOND,Clinique Communautaire de Martissant,PAP,Santé,2014-11-13,None,None,...,6,less or equal to 15,1,less or equal to 5,,None,None,2015-01-20,Q1,2015


In [4]:
#Overall Data for the period
eid_Qi = eid[(eid['Year'] == year)] #& (eid['Quarter'] == period_1)]
eid_Qi.shape

#Santé Network
eid_Qi_st = eid[(eid['Year'] == year) & (eid['Quarter'] == period_1) & (eid['Network'] == 'Santé')]
eid_Qi_st.shape

#Infants not linked to mothers
eid_link = eid[(eid['Liaison_mere'] == 'no') & (eid['Year'] == year)]


print('Overall:', eid_Qi.shape[0], 'spécimens. '
      'Santé:', eid_Qi_st.shape[0], 'spécimens. '
     'Not_linked:', eid_link.shape[0], 'enfants non liés.')

Overall: 787 spécimens. Santé: 168 spécimens. Not_linked: 257 enfants non liés.


# Dataset Overall & Santé

In [5]:
#Exportation de toute la base avec la date d'aujourd'hui
eid.to_excel(f"EID_{str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", index = False)
eid_Qi.to_excel(f"EID_FY23_{str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", index = False)
eid_link.to_excel(f"Enfants_non_liés_FY23_{str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", index = False)
# eid.to_excel(f"EID_FY22_All_years_10242022.xlsx", index = False)

# PMTCT_EID / HEI_POS / ON_ARV Overall

In [6]:
#Overall
#-----------------------------------------------------------------------------------------------------------------------
#Nombre de spécimens par tranche d'âge
PT = pd.pivot_table(eid_Qi, values='Patient_code', index = 'tranche_age',
                   aggfunc = len, fill_value = 0, margins = True, margins_name ='Total')
PT.rename(columns = {'Patient_code' : 'Spécimens prélevés'}, inplace = True)

#Pourcentage
PTp = round(PT.div(PT.iloc[-1,:], axis = 1), 2)*100
PTp.rename(columns = {'Spécimens prélevés' : '% Spécimens prélevés'}, inplace = True)

# Nombre de positif
eid_positive = eid_Qi[eid_Qi['Result'] == 'ADN VIH-1 Detècté']
PT2 = eid_positive.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, 
                               margins = True, margins_name ='Total')
PT2.rename(columns = {'Patient_code' : 'Enfants positifs'}, inplace = True)

#Nombre sous ARV
eid_on_arv = eid_positive[eid_positive['on_arv'] == 'yes']
PT3 = eid_on_arv.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, 
                             margins = True, margins_name = 'Total')
PT3.rename(columns = {'Patient_code' : 'Sous ARV'}, inplace = True)

#Enfants sans résultat
eid_no_result = eid_Qi[(eid_Qi['Result'] != 'ADN VIH-1 Non-Detècté') & (eid_Qi['Result'] != 'ADN VIH-1 Detècté')]
PT4 = eid_no_result.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len
                                , margins = True, margins_name = 'Total')
PT4.rename(columns = {'Patient_code' : 'Sans résultat'}, inplace =True)

#Concaténation des données et tableau final
EID = PT.merge(PTp, on = 'tranche_age', how = 'left').merge(PT2, on = 'tranche_age', how = 'left').merge(PT3, on = 'tranche_age', how = 'left').merge(PT4, on = 'tranche_age', how = 'left')
EID.reset_index(inplace = True)
EID = EID.fillna(0)
EID

,tranche_age,Spécimens prélevés,% Spécimens prélevés,Enfants positifs,Sous ARV,Sans résultat
0,0_2,512,65.0,3,3,354
1,2_12,275,35.0,8,5,204
2,Total,787,100.0,11,8,558


# Children without test result

In [7]:
#Positive children not on ARV
#Remplacement des strings par '-1' pour des futures opérations
eid_no_result.loc[(eid_no_result['days_in_lab'] == 'not_in_lab'), 'days_in_lab'] = -1
eid_no_result['days_in_lab'] = pd.to_numeric(eid_no_result['days_in_lab'])
col = ['Patient_code', 'CODE LABO', 'hospital', 'Office', 'Network', 'date_of_birth', 'gender', 'arv_start', 'lab', 'date_blood_taken',
       'date_caris_received_sample', 'date_lab_received_sample', '#days_in_lab', 'Result', 'tranche_age', 'on_arv', 'pos_dead_abandon', 'death_date',
       'abandoned_date', 'pcr_result_date']

def in_lab(x):
    if x < 0:
        return 'à Caris'
    elif x <= 15:
        return '<= 15j'
    else:
        return '> 15j'

# Création de la colonne '#days_in_lab' avec comme réponses les éléments de la fonction précédente
eid_no_result['#days_in_lab'] = eid_no_result['days_in_lab'].map(in_lab)
eid_no_result.drop(['Code_mere', 'Liaison_mere','total_jours_Labo', 'from_specimen_to_result', 'from_specimen_to_caris',
       'from_caris_to_lab', 'from_lab_to_result', 'Nbres_jours_Labo', 'days_in_lab', 'days_between_caris_lab', 'Nbres_jours_Caris',
                   'Quarter', 'Year'], axis = 1, inplace = True)
eid_no_result = eid_no_result.reindex(columns = col)

#Exporting children without test result
eid_no_result.to_excel('EID_no_test_result_Q1FY23 '+ str(datetime.today().strftime('%Y-%m-%d')) + '.xlsx', index = False)
eid_no_result.shape

(558, 20)

### Days in lab

In [8]:
lab_day = eid_no_result.pivot_table(index = '#days_in_lab', values = 'Patient_code', aggfunc = len, margins = True, margins_name = 'Total')
lab_day.reset_index(inplace = True)
lab_day.rename(columns = {'Patient_code' : 'Nombre de spécimens',
                         '#days_in_lab':'Situation'}, inplace = True)
lab_day

,Situation,Nombre de spécimens
0,> 15j,494
1,à Caris,64
2,Total,558


In [9]:
#Date replacement for further analysis
eid_positive.death_date.replace({None : '0000-00-00'}, inplace = True)
eid_positive.abandoned_date.replace({None : '0000-00-00'}, inplace = True)

# Dead and abandoned children

In [10]:
#Dead children
dead_children = eid_positive[eid_positive['death_date'] != '0000-00-00']
dead_children

,Patient_code,Code_mere,Liaison_mere,CODE LABO,hospital,Office,Network,date_of_birth,gender,arv_start,...,days_in_lab,Nbres_jours_Labo,days_between_caris_lab,Nbres_jours_Caris,pos_dead_abandon,death_date,abandoned_date,pcr_result_date,Quarter,Year
25877,STM/CSPP/ST02218,STM/CSPP/ST01160,no,STM/CSPP,Centre de Sante Pierre Payen,PAP,Santé,2021-12-18,,None,...,17,greater than 15,0,less or equal to 5,yes,2022-12-09,0000-00-00,2022-11-17,Q1,2023
25879,ARC/SADA/ST01714,0/0/0,no,ARC/SADA,SADA Matheux,PAP,Santé,2022-07-26,Female,None,...,10,less or equal to 15,0,less or equal to 5,yes,2022-10-23,0000-00-00,2022-11-10,Q1,2023
25947,CAY/CDSL/ST02368,CAY/CDSL/ST02355,yes,CAY/CDSL,Centre de Sante Lumiere (FINCA),CAY,Santé,2022-08-05,Female,2022-11-09,...,9,less or equal to 15,5,less or equal to 5,yes,2022-12-22,0000-00-00,2022-11-04,Q1,2023


In [11]:
# dead_children.to_excel('Dead_children_FY22_10242022.xlsx', index = False)

In [12]:
#Abandoned children
abandoned_children = eid_positive[eid_positive['abandoned_date'] != '0000-00-00']
abandoned_children

,Patient_code,Code_mere,Liaison_mere,CODE LABO,hospital,Office,Network,date_of_birth,gender,arv_start,...,days_in_lab,Nbres_jours_Labo,days_between_caris_lab,Nbres_jours_Caris,pos_dead_abandon,death_date,abandoned_date,pcr_result_date,Quarter,Year


### Table for the dead children

In [13]:
death = dead_children[['Patient_code', 'date_of_birth', 'tranche_age','date_blood_taken','arv_start','death_date']]
death.rename(columns = {'tranche_age': 'age_range'}, inplace = True)
death

,Patient_code,date_of_birth,age_range,date_blood_taken,arv_start,death_date
25877,STM/CSPP/ST02218,2021-12-18,2_12,2022-10-11,None,2022-12-09
25879,ARC/SADA/ST01714,2022-07-26,2_12,2022-10-17,None,2022-10-23
25947,CAY/CDSL/ST02368,2022-08-05,2_12,2022-10-18,2022-11-09,2022-12-22


# Not on ARV _ Overall

In [14]:
NOA = eid_positive[eid_positive['on_arv'] != 'yes'][['Patient_code', 'Network', 'death_date', 'abandoned_date']]
NOA.reset_index(inplace = True)
NOA.drop(columns = ['index'], inplace = True)
NOA.death_date.replace({None:'0000-00-00'}, inplace = True)
NOA.abandoned_date.replace({None:'0000-00-00'}, inplace = True)
NOA

,Patient_code,Network,death_date,abandoned_date
0,STM/CSPP/ST02218,Santé,2022-12-09,0000-00-00
1,ARC/SADA/ST01714,Santé,2022-10-23,0000-00-00
2,JER/HSPC/ST00417,CDS,0000-00-00,0000-00-00


### Reasons why not on ARV

In [15]:
def explanation_death(x):
    if x == '0000-00-00':
        return ''
    elif x != '0000-00-00':
        return 'Décès'
    else:
        return 'Error'

def explanation_abandon(x):
    if x == '0000-00-00':
        return ''
    elif x != '0000-00-00':
        return 'Abandon'
    else:
        return 'Error'    
    
# Création de nouvelles colonnes pour explications
NOA['Explication_death'] = NOA['death_date'].map(explanation_death)
NOA['Explication_abandon'] = NOA['abandoned_date'].map(explanation_abandon)
NOA['Explication'] = NOA.Explication_death + NOA.Explication_abandon
NOA.drop(['Network', 'death_date', 'abandoned_date', 'Explication_death', 'Explication_abandon'], axis = 1, inplace =True)
NOA

,Patient_code,Explication
0,STM/CSPP/ST02218,Décès
1,ARC/SADA/ST01714,Décès
2,JER/HSPC/ST00417,


---

# Summary _ Microsoft Word Document

In [16]:
#Writing the report with Docx
today = f"{str(datetime.today().day)} {str(datetime.now().strftime('%B'))} {str(datetime.today().year)}"
# today = "31 Décembre 2022"
#Period for the report
period = f"{start_date} au {today}"

document = Document()

document.add_heading(f"Rapport EID_{str(datetime.today().strftime('%Y-%m-%d'))}", level = 0)

#Intro
document.add_paragraph('Bonjour à tous,')
p1 = document.add_paragraph(f"Je partage avec vous les résultats EID tirés du système ce {today} pour la période ")
p1.add_run(f"du {period} ({period_1}).").bold = True

p2 = document.add_paragraph()

#Overall data
p2.add_run(f'Tableau 1: Résultats ({start_date} au {today})').bold=True

#add 2 tables of result
table1 = document.add_table(EID.shape[0]+1, EID.shape[1])

table1.style = 'Light List Accent 1'

#add the EID header rows
for j in range(EID.shape[-1]):
    table1.cell(0,j).text = EID.columns[j]
    
#add the rest of EID data
for i in range(EID.shape[0]):
    for j in range(EID.shape[-1]):
        table1.cell(i+1, j).text = str(EID.values[i,j])

p3 = document.add_paragraph()
p3= document.add_paragraph()
p3.add_run('Tableau 2: Nombre de jours au labo').bold = True

table2 = document.add_table(lab_day.shape[0]+1, lab_day.shape[1])
table2.style = 'Light List Accent 1'

#add the days in lab header rows
for j in range(lab_day.shape[-1]):
    table2.cell(0,j).text = lab_day.columns[j]
    
#add the rest of lab day data
for i in range(lab_day.shape[0]):
    for j in range(lab_day.shape[-1]):
        table2.cell(i+1, j).text = str(lab_day.values[i,j])
        
#Comments on the results

p4 = document.add_paragraph()

p5 = document.add_paragraph('Pour la période du ' + period + ', ')
p5.add_run(str(EID.values[2,1])).bold = True
p5.add_run(' spécimens ont été collectés dont ')
p5.add_run(str(EID.values[0,2]) + '%').bold =True
p5.add_run(' sont dans la tranche d\'âge 0-2 mois.')

p6 = document.add_paragraph()
p6.add_run(f"Parmi ces {EID.values[2,1]} spécimens, ")


#---------------------------------------------------------------------------------------------------------------

#function for the number of positive children
def positive_children():
    if EID.values[2, 3] == 0:
        p6.add_run("aucun résultat positif n'est retourné jusque-là.")
        p6.style = 'List Bullet'
    elif EID.values[2, 3] == 1 and EID.values[2, 4] != 0:
        p6.add_run("un enfant est dépisté positif et est placé sous ARV.").bold = True
        p6.style = 'List Bullet'        
    elif EID.values[2, 3] == 1 and EID.values[2, 4] == 0:
        p6.add_run("un enfant est dépisté positif mais n'est pas encore placé sous ARV.").bold = True
        p6.style = 'List Bullet'        
    elif EID.values[2, 3] > 1 and EID.values[2, 4] == 0:
        p6.add_run(f"{EID.values[2, 3]} enfants sont dépistés positifs mais aucun d'entre eux n'est placé sous ARV.").bold = True
        p6.style = 'List Bullet'
    elif EID.values[2, 3] > 1 and EID.values[2, 4] == 1:
        p6.add_run(f"{EID.values[2,3]} enfants sont dépistés positifs, dont {EID.values[2,4]} est placé sous ARV").bold = True
        p6.add_run(', soit ')
        Percent_on_arv = round((EID.values[2,4]/EID.values[2,3])*100, 2)
        run = p6.add_run(f"{Percent_on_arv}%")
        font = run.font
        font.bold = True
        red = RGBColor(255, 0, 0)
        font.color.rgb = red
        p6.add_run(' des cas positifs.')
        p6.style = 'List Bullet'
    elif EID.values[2, 3] > 1 and EID.values[2, 4] != 0 and EID.values[2, 3] == EID.values[2, 4]:
        p6.add_run(f"{EID.values[2,3]} enfants sont dépistés positifs, et les {EID.values[2,4]} sont placés sous ARV").bold = True
        p6.add_run(', soit ')
        Percent_on_arv = round((EID.values[2,4]/EID.values[2,3])*100, 2)
        run = p6.add_run(f"{Percent_on_arv}%")
        font = run.font
        font.bold = True
        red = RGBColor(0x00, 0x00, 0xFF)
        font.color.rgb = red
        p6.add_run(' des cas positifs.')
        p6.style = 'List Bullet'        
    elif EID.values[2, 3] > 1 and EID.values[2, 4] != 0:
        p6.add_run(f"{EID.values[2,3]} enfants sont dépistés positifs, dont {EID.values[2,4]} sont placés sous ARV").bold = True
        p6.add_run(', soit ')
        Percent_on_arv = round((EID.values[2,4]/EID.values[2,3])*100, 2)
        run = p6.add_run(f"{Percent_on_arv}%")
        font = run.font
        font.bold = True
        red = RGBColor(255, 0, 0)
        font.color.rgb = red
        p6.add_run(' des cas positifs.')
        p6.style = 'List Bullet'
#Calling the positive_children function
positive_children()
#--------------------------------------------------------------------------------------------------------------------


#Number of days in lab
#--------------------------------------------------------------------------------------------------------------------
#slice of the lab_day table
lab_day2 = lab_day.values[0:,:]

#variable containing the number of specimens in Caris
in_caris = 0
for elt in lab_day2:
    if 'à Caris' in elt:
        in_caris = elt[-1]

#variable containing the number of specimens with more than 15 days in lab
more_15 = 0
for item in lab_day2:
    if '> 15j' in item:
        more_15 = item[-1]


#function for the number of days in lab

def days_in_lab():
    if EID.values[2, 1] != 0 and EID.values[2, 5] == 0:
        p7 = document.add_paragraph()
        p7.add_run('Tous les enfants testés ont un résultat.')
        p7.style = 'List Bullet'
    elif EID.values[2, 5] != 0 and '> 15j' not in lab_day.values[:, 0] and 'à Caris' not in lab_day.values[:, 0]:
        p7 = document.add_paragraph()
        p7.add_run(str(EID.values[2, 5])).bold = True
        p7.add_run(' spécimens n\'ont pas encore de résultat. Tous ces spécimens ont moins de 15 jours au laboratoire.')
        p7.style = 'Liste Bullet'
    elif EID.values[2, 5] != 0 and '> 15j' not in lab_day.values[:, 0] and 'à Caris' in lab_day.values[:, 0]:
        if in_caris == 1:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {in_caris} seul est au niveau de Caris.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'
        else:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {in_caris} sont au niveau de Caris.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'
    elif EID.values[2, 5] != 0 and '> 15j' in lab_day.values[:, 0] and 'à Caris' in lab_day.values[:, 0]:
        if in_caris == 1 and more_15 == 1:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {more_15} a plus de 15 jours au labo et {in_caris} seul est au niveau de Caris.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'
        elif in_caris > 1 and more_15 == 1:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {more_15} a plus de 15 jours au labo et {in_caris} sont au niveau de Caris.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'        
        elif in_caris == 1 and more_15 > 1:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {more_15} ont plus de 15 jours au labo et {in_caris} seul est au niveau de Caris.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'        
        else:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {more_15} ont plus de 15 jours au labo et {in_caris} sont au niveau de Caris.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'
    else:
        if more_15 == 1:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {more_15} a plus de 15 jours au labo.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'
        else:
            p7 = document.add_paragraph()
            p7.add_run(str(EID.values[2, 5])).bold = True
            p7.add_run(' spécimens n\'ont pas encore de résultat. ')
            run2 = p7.add_run(f"Parmi ces {EID.values[2, 5]} spécimens sans résultat, {more_15} ont plus de 15 jours au labo.")
            font2 = run2.font
            font2.bold = True
            font2.highlight_color = WD_COLOR_INDEX.YELLOW
            p7.style = 'List Bullet'

#Calling the days_in_lab function
days_in_lab()

#---------------------------------------------------------------------------------------------------------------------
#function for the dead and abandoned children

p8 = document.add_paragraph()
def lost_children():
    if len(dead_children.Patient_code) == 0 and len(abandoned_children.Patient_code) == 0:
        p8.add_run("Aucun cas de décès ni d'abandon n'a été enregistré jusque-là.")
        p8.style = 'List Bullet'
    elif len(dead_children.Patient_code) != 0 and len(abandoned_children.Patient_code) == 0:
        p8.add_run("Parmi les enfants positifs, nous avons enregistré ")
        p8.add_run(f"{len(dead_children.Patient_code)} cas de décès {tuple(dead_children.Patient_code)}.").bold = True
        p8.style = 'List Bullet'        
    elif len(dead_children.Patient_code) == 0 and len(abandoned_children.Patient_code) != 0:
        p8.add_run("Parmi les enfants positifs, nous avons enregistré ")
        p8.add_run(f"{len(abandoned_children.Patient_code)} cas d'abandon {tuple(abandoned_children.Patient_code)}.").bold = True
        p8.style = 'List Bullet'
    else:
        p8.add_run("Parmi les enfants positifs, nous avons enregistré ")
        p8.add_run(f"{len(dead_children.Patient_code)} cas de décès {tuple(dead_children.Patient_code)}, et {len(abandoned_children.Patient_code)} cas d'abandon {tuple(abandoned_children.Patient_code)}.").bold = True
        p8.style = 'List Bullet'
        

#--------------------------------------------------------------------------------------------------------------------

#Calling the lost_children function
lost_children()

p999 = document.add_paragraph()

#--------------------------------------------------------------------------------------------------------------------
#function for dead children

def death_table():
    if len(death.Patient_code) != 0:        
        p99 = document.add_paragraph()
        p99.add_run("Tableau 3: Informations sur les enfants décédés").bold = True
        table3 = document.add_table(death.shape[0]+1, death.shape[1])
        table3.style = 'Light List Accent 1'

        #add the days in lab header rows
        for j in range(death.shape[-1]):
            table3.cell(0,j).text = death.columns[j]

        #add the rest of lab day data
        for i in range(death.shape[0]):
            for j in range(death.shape[-1]):
                table3.cell(i+1, j).text = str(death.values[i,j])

        p9 = document.add_paragraph()

#Calling the death_table function
death_table()

#---------------------------------------------------------------------------------------------------------------------
#function for children not on ARV

def not_on_arv():
    if len(NOA.Patient_code) != 0:
        p10 = document.add_paragraph()
        p10.add_run('Tableau 4: Liste des enfants positifs non sous ARV').bold =True

        table4 = document.add_table(NOA.shape[0]+1, NOA.shape[1])
        table4.style = 'Light List Accent 1'

        for j in range(NOA.shape[-1]):
            table4.cell(0,j).text = NOA.columns[j]

        for i in range(NOA.shape[0]):
            for j in range(NOA.shape[-1]):
                table4.cell(i+1,j).text = str(NOA.values[i,j])

        p111 = document.add_paragraph()

#Calling the not_on_arv function
not_on_arv()

#----------------------------------------------------------------------------------------------------------------------        

#Conclusion

p15 = document.add_paragraph()
p15.add_run('Merci pour les suivis que vous allez assurer.').italic = True
p16 = document.add_paragraph('Cordialement,')
document.save(f"EID_report_{str(datetime.today().strftime('%Y-%m-%d'))}_Q1FY23.docx")

                                                                                                        THE END